In [103]:
% matplotlib notebook
import cirq as cirq
import numpy as np
import scipy as sp
import scipy.linalg as la
from xmps.spin import paulis
from scipy.linalg import norm
X, Y, Z = paulis(0.5)

class LeftFullGate(cirq.Gate):
    """
    Parametrisation of tr 1 non unitary on right hand site (gate goes on left of diagram). 
    Singular values are np.array([cos(a), sin(a)])"""
    def __init__(self, params, D=2):
        self.params = params
    def _decompose_(self, qubits):
        a, b, c, d, e, f = self.params
        return [cirq.Rx(a*2)(qubits[0]), cirq.inverse(cirq.S(qubits[0])),
                cirq.Rz(b)(qubits[1]), cirq.Rx(c)(qubits[1]), cirq.Rz(d)(qubits[1]),
                cirq.CNOT(qubits[1], qubits[0]),
                cirq.Rx(f)(qubits[1]), cirq.Rz(e)(qubits[1])] 
    def singular_values(self):
        return np.abs(np.array([np.cos(self.params[0]), np.sin(self.params[0])]))
    def num_qubits(self):
        return 2
    
class RightFullGate(cirq.Gate):
    """
    Parametrisation of tr 1 non unitary on left hand site (gate goes on right of diagram). 
    Singular values are np.array([np.sqrt((1-np.sin(a)*np.sin(b))/2), np.sqrt((1+np.sin(a)*np.sin(b))/2)])"""
    def __init__(self, params, D=2):
        self.params = params
    def _decompose_(self, qubits):
        return [cirq.SWAP(*qubits),
                cirq.inverse(LeftFullGate(self.params)(*qubits)),
                cirq.SWAP(*qubits)] 
    def singular_values(self):
        return np.abs(np.array([np.cos(self.params[0]), np.sin(self.params[0])]))
    
    def num_qubits(self):
        return 2

In [121]:
qbs = cirq.LineQubit.range(2)
C = cirq.Circuit.from_ops(cirq.decompose_once(LeftFullGate(np.random.randn(6))(*qbs)))
a, b, c, d, e, f = x = np.random.randn(6)
U1 = cirq.unitary(LeftFullGate(x))
U2 = cirq.unitary(RightFullGate(x))

z = np.array([1, 0])
ρ1 = np.tensordot(np.tensordot(U1.reshape(2, 2, 2, 2), z, [-2, 0]), z, [0, 0])
ρ2 = np.tensordot(np.tensordot(U2.reshape(2, 2, 2, 2), z, [-1, 0]), z, [1, 0])
u1, s1, v1 = np.linalg.svd(ρ1)
u2, s2, v2 = np.linalg.svd(ρ2)

s1, [np.cos(a), np.sin(a)], s2
assert np.allclose(ρ1, ρ2.conj().T)

[[ 0.01109451-0.15171059j -0.35918991-0.55430511j]
 [ 0.0721127 -0.11377644j  0.04684822+0.72128301j]] 
 [[ 0.01109451+0.15171059j  0.0721127 +0.11377644j]
 [-0.35918991+0.55430511j  0.04684822-0.72128301j]]


In [14]:
np.arctan2(-1, 1)/np.pi

-0.25

In [58]:
print(C.to_text_diagram(transpose=True))

0          1
│          │
Rx(0.855π) Rz(0.063π)
│          │
S^-1       Rx(0.354π)
│          │
│          Rz(0.229π)
│          │
X──────────@
│          │
│          Rx(-0.13π)
│          │
│          Rz(0.158π)
│          │


In [119]:
import sympy as sp
a, b, c, d, e, f, g, h = sp.symbols('a, b, c, d, e, f, g, h')
A = sp.Matrix([[a, b], [c, d]])
B = sp.Matrix([[e, f], [g, h]])
def op(r):
    return A*r*A.H+B*r*B.H

In [120]:
op(sp.Matrix([[1, 0], [0, 0]]))

Matrix([
[a*conjugate(a) + e*conjugate(e), a*conjugate(c) + e*conjugate(g)],
[c*conjugate(a) + g*conjugate(e), c*conjugate(c) + g*conjugate(g)]])